In [61]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

In [62]:
from preprocessing import *
data_path = "data/game.csv"
X_train, X_test, y_train, y_test = prep_all(data_path)
# y_test.drop()

In [63]:
# from sklearn.linear_model import LogisticRegression
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import GridSearchCV
# from sklearn.svm import SVC
# from sklearn.neural_network import MLPClassifier

# # Define your models and their hyperparameters
# models = [
#     ('Logistic Regression', LogisticRegression(), {'C': [0.1, 1, 10]}),
#     ('Random Forest', RandomForestClassifier(), {'n_estimators': [10, 50, 100], 'max_depth': [None, 10, 20]}),
#     ('SVM', SVC(), {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}),
#     ('Neural Network', MLPClassifier(), {'hidden_layer_sizes': [(10,), (50,), (100,)], 'activation': ['relu', 'tanh']})
# ]

# # Use cross-validation and grid search to compare the models
# for name, model, params in models:
#     grid_search = GridSearchCV(model, params, cv=5, scoring='accuracy')
#     grid_search.fit(X_train, y_train)
    
#     print(f"{name} Best Parameters: {grid_search.best_params_}")
#     print(f"{name} Best Score: {grid_search.best_score_}")


In [64]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import optuna

In [65]:
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32).unsqueeze(1)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long).view(-1, 1)

X_val_tensor = torch.tensor(X_test.values, dtype=torch.float32).unsqueeze(1)
y_val_tensor = torch.tensor(y_test.values, dtype=torch.long).view(-1, 1)

dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
train_loader = DataLoader(dataset, batch_size=64, shuffle=True)
valid_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)


In [66]:
import torch
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # Set initial hidden and cell states
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

        # Forward propagate RNN
        out, _ = self.rnn(x, h0)

        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out


In [74]:
input_size = 136
output_size = 2
device = torch.device("mps")
hidden_size = 32
num_layers = 2
learning_rate = 0.001
num_epochs = 10
model = RNN(input_size, hidden_size, output_size, num_layers).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

def train(model, train_loader, optimizer, criterion):
    model.train()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        inputs = inputs.to(device)
        labels = labels.to(device).view(-1)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    train_loss = running_loss / len(train_loader)
    return train_loss

train(model, train_loader, optimizer, criterion)

0.6731668484702794

In [75]:
def objective(trial):
    # Define hyperparameters
    hidden_size = trial.suggest_int("hidden_size", 16, 512)
    num_layers = trial.suggest_int("num_layers", 1, 3)
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 1e-2)

    # Create the RNN model
    model = RNN(input_size, hidden_size, output_size, num_layers).to(device)

    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Train the model
    num_epochs = 50
    for epoch in range(num_epochs):
        model.train()
        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device).view(-1)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

        # Evaluate the model
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, targets in valid_loader:
                inputs, targets = inputs.to(device), targets.to(device).view(-1)
                outputs = model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += targets.size(0)
                correct += (predicted == targets).sum().item()

        accuracy = correct / total
        trial.report(accuracy, epoch)

        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return accuracy

In [76]:
# Specify the number of trials
num_trials = 2000
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=num_trials)

# Print the best trial
best_trial = study.best_trial
print(f"Best trial: {best_trial.number}, accuracy: {best_trial.value}")
print(f"Best hyperparameters: {best_trial.params}")


[I 2023-04-28 20:58:23,350] A new study created in memory with name: no-name-3d91714e-45c7-49bb-90bc-a0ea42874064
/var/folders/29/bwn4vhms0yd66vmmfws1d2800000gn/T/ipykernel_49249/415702916.py:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 1e-2)
[I 2023-04-28 21:06:12,121] Trial 0 finished with value: 0.5998769987699877 and parameters: {'hidden_size': 140, 'num_layers': 2, 'learning_rate': 0.002871658180989135}. Best is trial 0 with value: 0.5998769987699877.
[W 2023-04-28 21:07:26,399] Trial 1 failed with parameters: {'hidden_size': 222, 'num_layers': 1, 'learning_rate': 8.29731150624263e-05} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/Users/spencer/opt/anaconda3/envs/pytorch/lib/python3.10/site-pa

KeyboardInterrupt: 